In [1]:
import sys, os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import collections

sys.path.insert(0, os.path.dirname(os.path.abspath('../src')))
from src.getjobsch import *

### Web Scraping

The file `src/getjobsch` contains the necesary functions to pull infomation from https://www.jobs.ch/en/vacancies/. The function works in the following way:
- Receives a list of job positions on natural language
- The function `clean_job_keywords` will transform those key words to search keywords by removing white spaces and replacing them with `%20` characters
- Once the necesary keywords were obtained the function `df_full_data` will proceed to pull info for each job in the following way:
  - Get the number of available pages for each job position
  - For each of the available pages, scrap an individual text box using the function `get_data_one_job` and concatenating the info by using the function `df_all_jobs`
  - In case no job postings are found an error should be printed (see example below).

In [ ]:
# Key words to be searched
job_positions = ["Data Engineer", "Data Scientist", "Data Analyst", "Python Developer", "Data Manager", "Data Architect", "Big Data Analyst", "Data Python"]

In [ ]:
# DO NOT RUN

# Run the function to get both errors and 
df_all = df_full_data(job_positions)

# In this case we should not have errors
errors = df_all["errors"]
errors

In [ ]:
# Print the found jobs
df_jobs = df_all["results"]
df_jobs.head(10)

In case of not web scraping load the data

In [40]:
# Load found jobs
# df_jobs = pd.read_csv("../data/raw/df_jobs_ch.csv", index_col=[0])
# df_jobs.head()

,title,publication_date,location,workload,job_type,company,job_link,keyword
0,Data Engineer temp. 24 months (w/m/d),25 avril 2023,Baden,100%,Temporary,Axpo Group,https://www.jobs.ch/en/vacancies/detail/3fa23b...,data engineer
1,Big Data Engineer (w/m/d),24 mai 2023,St. Gallen,80% – 100%,Unlimited employment,Raiffeisen Schweiz,https://www.jobs.ch/en/vacancies/detail/450b00...,data engineer
2,CSV Engineer MedTech 80% (5306 – KFR),17 mai 2023,Ostschweiz,80%,Temporary,CTC Resourcing Solutions,https://www.jobs.ch/en/vacancies/detail/cf1d02...,data engineer
3,Azure Data Ingenieur (m/w/d),23 mai 2023,Zell LU,80% – 100%,Unlimited employment,ROCKEN,https://www.jobs.ch/en/vacancies/detail/7720c6...,data engineer
4,Low Code Solution Engineer (80-100 %),15 mai 2023,Ittigen,80% – 100%,Unlimited employment,Gebäudeversicherung Bern – GVB Gruppe,https://www.jobs.ch/en/vacancies/detail/96ba41...,data engineer


## Web scrapping the individual job positions

In [3]:
## DO NOT RUN

results = get_job_keywords(df_jobs)

In [4]:
programming_summary = results["programming_summary"]
skills_summary = results["skills_summary"]
python_summary = results["python_summary"]
errors = results["errors"]

In [5]:
print(f"There were {len(errors)} positions without available information")

There were 452 positions without available information


In [6]:
print(programming_summary)

Counter({'python': 2208, 'sql': 1810, 'r': 517, 'javascript': 239, 'php': 205, 'java': 192, 'plsql': 170, 'scala': 121, 'matlab': 116, 'powershell': 115, 'perl': 102, 'c++': 87, 'pascal': 87, 'bash': 82, 'vba': 68, 'nosql': 48, 'c#': 42, 'go': 42, 'css': 42, 'mssql': 34, 'tsql': 34, 'ruby': 34, 'assembly': 28, 'html': 21, 'typescript': 14})


In [7]:
print(python_summary)

Counter({'pytorch': 155, 'pandas': 68, 'scikitlearn': 68, 'tensorflow': 68, 'plotly': 14})


In [8]:
print(skills_summary)

Counter({'sap': 2352, 'aws': 732, 'tableau': 565, 'excel': 549, 'hadoop': 526, 'spark': 364, 'linux': 326, 'powerbi': 325, 'airflow': 232, 'sharepoint': 123, 'qlik': 82, 'pandas': 68, 'spss': 68, 'word': 34, 'powerpoint': 34, 'vue': 34, 'ssrs': 34, 'ssis': 14, 'jquery': 7})


### Store Raw Data

In [9]:
# df_jobs.to_csv("../data/raw/df_jobs_ch.csv")
# pd.DataFrame(dict(programming_summary).items()).to_csv("../data/raw/programming_summary_2.csv")
# pd.DataFrame(dict(python_summary).items()).to_csv("../data/raw/python_summary_2.csv")
# pd.DataFrame(dict(skills_summary).items()).to_csv("../data/raw/skills_summary_2.csv")

## Grouped data web scrapping

Here we applied the `get_job_keywords` to the grouped data set. This will be migrated to a function in `src`.

In [38]:
df_reduced = df_jobs[df_jobs["keyword"].isin(["data engineer", "data scientist", "data analyst"])].copy()
df_reduced.head()

,title,publication_date,location,workload,job_type,company,job_link,keyword
0,Data Engineer temp. 24 months (w/m/d),25 avril 2023,Baden,100%,Temporary,Axpo Group,https://www.jobs.ch/en/vacancies/detail/3fa23b...,data engineer
1,Big Data Engineer (w/m/d),24 mai 2023,St. Gallen,80% – 100%,Unlimited employment,Raiffeisen Schweiz,https://www.jobs.ch/en/vacancies/detail/450b00...,data engineer
2,CSV Engineer MedTech 80% (5306 – KFR),17 mai 2023,Ostschweiz,80%,Temporary,CTC Resourcing Solutions,https://www.jobs.ch/en/vacancies/detail/cf1d02...,data engineer
3,Azure Data Ingenieur (m/w/d),23 mai 2023,Zell LU,80% – 100%,Unlimited employment,ROCKEN,https://www.jobs.ch/en/vacancies/detail/7720c6...,data engineer
4,Low Code Solution Engineer (80-100 %),15 mai 2023,Ittigen,80% – 100%,Unlimited employment,Gebäudeversicherung Bern – GVB Gruppe,https://www.jobs.ch/en/vacancies/detail/96ba41...,data engineer


Here we web scraped the grouped data

In [12]:
## DO NOT RUN

grouped_results = {}
for i, (grp, df) in enumerate(df_reduced.groupby(by = "keyword")):
  g_results = get_job_keywords(df)
  grouped_results[grp] = grouped_results.get(grp, g_results)

In [13]:
grouped_results.keys()

dict_keys(['data analyst', 'data engineer', 'data scientist'])

In [23]:
grouped_results["data scientist"].keys()

dict_keys(['programming_summary', 'skills_summary', 'python_summary', 'errors'])

In [28]:
ds_prog = pd.DataFrame(dict(grouped_results["data scientist"]["programming_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
ds_prog["type"] = "programming"

ds_tech = pd.DataFrame(dict(grouped_results["data scientist"]["skills_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
ds_tech["type"] = "tech"

ds_python = pd.DataFrame(dict(grouped_results["data scientist"]["python_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
ds_python["type"] = "python"

df_ds = pd.concat([ds_prog, ds_tech, ds_python], ignore_index=True) # .reset_index()
df_ds["keyword"] = "data scientist"
# df_ds

In [31]:
de_prog = pd.DataFrame(dict(grouped_results["data engineer"]["programming_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
de_prog["type"] = "programming"

de_tech = pd.DataFrame(dict(grouped_results["data engineer"]["skills_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
de_tech["type"] = "tech"

de_python = pd.DataFrame(dict(grouped_results["data engineer"]["python_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
de_python["type"] = "python"

df_de = pd.concat([de_prog, de_tech, de_python], ignore_index=True) # .reset_index()
df_de["keyword"] = "data engineer"
# df_de

In [33]:
da_prog = pd.DataFrame(dict(grouped_results["data analyst"]["programming_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
da_prog["type"] = "programming"

da_tech = pd.DataFrame(dict(grouped_results["data analyst"]["skills_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
da_tech["type"] = "tech"

da_python = pd.DataFrame(dict(grouped_results["data analyst"]["python_summary"]).items()).rename(columns={0 : "skill", 1 : "count"})
da_python["type"] = "python"

df_da = pd.concat([da_prog, da_tech, da_python], ignore_index=True) # .reset_index()
df_da["keyword"] = "data analyst"
# df_da

Writing the data to a csv file, for faster plotting.

In [35]:
df_all = pd.concat([df_ds, df_de, df_da], ignore_index = True)
# df_all.to_csv("../data/raw/skills_by_category.csv")

In [39]:
df_all.head()

,skill,count,type,keyword
0,sql,300,programming,data scientist
1,python,200,programming,data scientist
2,r,150,programming,data scientist
3,javascript,50,programming,data scientist
4,php,25,programming,data scientist
